# Imports

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from os import listdir
import pathlib

import pickle

In [2]:
# Define functions

# Prepare ABT table according to specific dimension
def abt(org_df,dimension):
    '''
    Transform the original data set into ABT table by specific dimension provided in 2nd argument.
    By customer, then input string "DestName" in dimension.
    By region, then input string "DestZone2"
    '''
    # Count number of truck by truck type
    cnt_of_trucks = pd.pivot_table(org_df,values=['SHIPMENT_XID'],columns=['FIRST_EQUIPMENT_GROUP_GID'],
                    index = ['start_time',dimension],fill_value=0,
                    aggfunc=lambda x: len(x.unique()))
    # Simplify truck type names
    cnt_col_name = []
    for i in cnt_of_trucks.columns:
        cnt_col_name.append(i[1][i[1].index('.')+1:])
    cnt_of_trucks.columns = cnt_col_name

    cnt_of_trucks.reset_index(level = dimension,inplace=True, col_level=1)
    # Aggregate volume by Customer
    df_vol = pd.pivot_table(org_df,values=['TOTAL_VOLUME'],
                    index = ['start_time',dimension],fill_value=0,aggfunc=np.sum)

    df_vol.reset_index(level = dimension,inplace=True)
    df_vol['Date']=df_vol.index
    df_vol['Year']=df_vol.Date.apply(lambda x: x.year)
    df_vol['Month']=df_vol.Date.apply(lambda x:x.month)
    df_vol['Weekday']=df_vol.Date.apply(lambda x:x.weekday()+1)


    # Concatenate volume and count of trucks
    df_abt = pd.concat([df_vol,cnt_of_trucks],axis=1)
    df_abt.drop(axis=1, labels=['Date'],inplace=True)
    df_abt = df_abt.loc[:,~df_abt.columns.duplicated()]
    return df_abt

# Settings

In [3]:
# Number of days to forecast
number_of_day = 7

# Directory where the data is located
data_path = 'data/'

# Directory where the models are located
model_path = 'models/'

# List of IMT regions
IMT_list = ['AEON BIG (M) SDN. BHD - DC',
            'AEON DC',
            'GCH DISTRIBUTION CENTRE- SEPANG',
            'GCH RETAIL (MALAYSIA) SDN BHD',
            'MYDIN CENTRAL DISTRIBUTION CENTRE - CDC',
            'TESCO GROCERY (AMBIENT DC)',
            'WATSON',
            'TESCO DC(AMBIENT DC)',
            'JAYA GROCER DISTRIBUTION CENTER']

# List of DT regions
DT_list = ['SOUTH', 
           'NORTH', 
           'CENTRAL',
           'EAST COAST']

# Load Data

In [4]:
# Load CSV
df = pd.read_csv(data_path+'input_from_SQL_query.csv')

In [5]:
# Reformat float number into 2 decimals and reformat start_time format
i = 0
for j in df.dtypes:
    if j == np.float64:
        df.iloc[:,i] = np.round(df.iloc[:,i],decimals=2)
        i += 1
    else:
        i += 1
        continue

df['start_time'] = pd.to_datetime(df['START_TIME']).dt.date

In [6]:
df

,SHIPMENT_XID,START_TIME,TOTAL_SHIP_UNIT_COUNT,TOTAL_WEIGHT,TOTAL_VOLUME,FIRST_EQUIPMENT_GROUP_GID,PRINCIPAL,DestName,Region,City,DestZone1,DestZone2,Segment,start_time
0,328440,2018-11-26 02:37:44,60,15000.00,19.38,LFL/TMS/MYS.20T_BOX,UNILEVER,JULIE'S MANUFACTURING SDN BHD,MELAKA,MELAKA,MLK-001,SOUTH,DT-OUTSTATION,2018-11-26
1,386949,2019-06-14 07:00:00,343,3183.41,6.35,LFL/TMS/MYS.10T_BOX,UNILEVER,SIMA FROZEN PRODUCTS SDN BHD,KEDAH,ALOR SETAR,KDH-001,NORTH,DT-OUTSTATION,2019-06-14
2,328507,2018-11-16 16:01:00,450,5682.13,15.25,LFL/TMS/MYS.20T_BOX,UNILEVER,WATSON,SELANGOR,SHAH ALAM,KLV-001,CENTRAL,IMT-NS,2018-11-16
3,337436,2018-12-11 07:00:00,681,6158.84,15.24,LFL/TMS/MYS.10T_BOX,UNILEVER,BOSTON TRADING SDN BHD,MELAKA,BATU BERENDAM,MLK-003,SOUTH,DT-OUTSTATION,2018-12-11
4,337321,2018-12-16 23:00:00,645,5538.75,14.98,LFL/TMS/MYS.20T_BOX,UNILEVER,STP CHILLED AND FROZEN PRODUCTS SDN BHD,SELANGOR,SHAH ALAM,KLV-007,CENTRAL,DT-LOCAL,2018-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24851,567405,2020-11-23 07:00:00,716,8700.79,22.42,LFL/TMS/MYS.20T_BOX,UNILEVER,SIN GUAN HENG PROVISIONS SDN. BHD,PERAK,PARIT BUNTAR,PNG-008,NORTH,DT-OUTSTATION,2020-11-23
24852,567403,2020-11-23 07:00:00,1453,10938.22,28.72,LFL/TMS/MYS.20T_BOX,UNILEVER,JEMENTAH SOON HUAT TRADING SDN BHD,JOHOR,JEMENTAH,JNT-002,SOUTH,DT-OUTSTATION,2020-11-23
24853,567816,2020-11-23 11:00:00,260,4498.60,11.96,LFL/TMS/MYS.20T_BOX,UNILEVER,AEON DC,SELANGOR,BANDAR BARU BANGI,KLV-003,CENTRAL,IMT-NS,2020-11-23
24854,567813,2020-11-23 11:00:00,426,5378.76,13.09,LFL/TMS/MYS.20T_BOX,UNILEVER,AEON DC,SELANGOR,BANDAR BARU BANGI,KLV-003,CENTRAL,IMT-NS,2020-11-23


# IMT (By Customer)

## Preparing IMT Data

In [7]:
# Group fleet data for IMT
IMT = df[df.Segment=='IMT']
IMT_NS = df[df.Segment=='IMT-NS']
df_imt = IMT.append(IMT_NS)

In [8]:
# Keep top 4 frequently used truck types
index_to_remove = []
for i in list(pd.value_counts(df_imt.FIRST_EQUIPMENT_GROUP_GID).index)[4:]:
    index_to_remove += list(df_imt[df_imt.FIRST_EQUIPMENT_GROUP_GID == i].index)

df_imt.drop(axis=0, labels=index_to_remove, inplace=True)

In [9]:
# Replace "WATSON DC" with "WATSON"
temp_index = list(df_imt[df_imt.DestName == 'WATSON DC'].index)
for i in temp_index:
    df_imt.DestName.loc[i] = 'WATSON'

In [10]:
# Aggregate IMT volume by customer
df_imt_vol = pd.pivot_table(df_imt, values=['TOTAL_VOLUME'], index=['DestName'], aggfunc=np.sum, fill_value=0)
df_imt_vol.sort_values(by=['TOTAL_VOLUME'], ascending=False, inplace=True)
remove_destname_list = list(df_imt_vol.index)[9:]  # Top 9 DestName

In [11]:
# Remove DestName 'AEON BIG 3RD ...' and 'AEON BIG 2ND DC ...' because these are ad-hoc shipments
index_to_remove = []
for i in remove_destname_list:
    index_to_remove += list(df_imt[df_imt.DestName == i].index)
df_imt.drop(axis=0, labels=index_to_remove, inplace=True)

In [12]:
# IMT ABT (Analytics Base Table)
df_imt_abt = abt(df_imt,"DestName")

In [13]:
df_imt_abt

,DestName,TOTAL_VOLUME,Year,Month,Weekday,10T_BOX,1T,20T_BOX,3T
start_time,,,,,,,,,
2018-10-25,AEON BIG (M) SDN. BHD - DC,0.10,2018,10,4,0,1,0,0
2018-10-25,AEON DC,47.64,2018,10,4,0,0,3,0
2018-10-25,GCH DISTRIBUTION CENTRE- SEPANG,0.73,2018,10,4,0,1,0,0
2018-10-25,GCH RETAIL (MALAYSIA) SDN BHD,11.81,2018,10,4,0,0,1,0
2018-10-25,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC,15.79,2018,10,4,0,0,1,0
...,...,...,...,...,...,...,...,...,...
2020-11-23,AEON BIG (M) SDN. BHD - DC,12.02,2020,11,1,0,1,1,0
2020-11-23,AEON DC,72.91,2020,11,1,1,0,5,1
2020-11-23,GCH DISTRIBUTION CENTRE- SEPANG,13.68,2020,11,1,0,1,1,0


In [14]:
IMT_list

['AEON BIG (M) SDN. BHD - DC',
 'AEON DC',
 'GCH DISTRIBUTION CENTRE- SEPANG',
 'GCH RETAIL (MALAYSIA) SDN BHD',
 'MYDIN CENTRAL DISTRIBUTION CENTRE - CDC',
 'TESCO GROCERY (AMBIENT DC)',
 'WATSON',
 'TESCO DC(AMBIENT DC)',
 'JAYA GROCER DISTRIBUTION CENTER']

## Load Pre-Trained Models

In [15]:
# Load pre-trained IMT models
IMT_model = {}
for dest in IMT_list:
    IMT_model[dest] = pickle.load(open('models/imt_'+dest+'.pkl', 'rb'))

In [16]:
# Use lastest few days as input for forecast 
imt_input = df_imt_abt[df_imt_abt.columns[:-4]].tail(number_of_day*9)

In [17]:
imt_input

,DestName,TOTAL_VOLUME,Year,Month,Weekday
start_time,,,,,
2020-11-06,GCH DISTRIBUTION CENTRE- SEPANG,100.05,2020,11,5
2020-11-06,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC,169.75,2020,11,5
2020-11-06,WATSON,35.28,2020,11,5
2020-11-08,GCH DISTRIBUTION CENTRE- SEPANG,1.45,2020,11,7
2020-11-08,GCH RETAIL (MALAYSIA) SDN BHD,6.53,2020,11,7
...,...,...,...,...,...
2020-11-23,AEON BIG (M) SDN. BHD - DC,12.02,2020,11,1
2020-11-23,AEON DC,72.91,2020,11,1
2020-11-23,GCH DISTRIBUTION CENTRE- SEPANG,13.68,2020,11,1


In [19]:
output_imt = pd.DataFrame(index=imt_input.index.unique())
for dest in IMT_list:
    try:
        temp_df = imt_input[imt_input['DestName'] == dest]
        temp_df = temp_df[temp_df.columns[-4:]]
        temp_output = IMT_model[dest].predict(np.array(temp_df))

        temp_col_name = []
        for col in df_imt_abt.columns[-4:]:
            temp_col_name.append(dest+'-'+col)
        temp_output = pd.DataFrame(np.round(temp_output), 
                                   index=temp_df.index,
                                   columns=temp_col_name)
        temp_output.astype(np.int64)
        output_imt = output_imt.join(temp_output, how='left')
        display(temp_output)
    except:
        pass

,AEON BIG (M) SDN. BHD - DC-10T_BOX,AEON BIG (M) SDN. BHD - DC-1T,AEON BIG (M) SDN. BHD - DC-20T_BOX,AEON BIG (M) SDN. BHD - DC-3T
start_time,,,,
2020-11-09,0.0,0.0,3.0,0.0
2020-11-11,0.0,0.0,3.0,0.0
2020-11-12,0.0,1.0,3.0,0.0
2020-11-16,0.0,0.0,3.0,0.0
2020-11-18,0.0,0.0,15.0,0.0
2020-11-19,0.0,1.0,0.0,0.0
2020-11-20,0.0,0.0,15.0,0.0
2020-11-23,0.0,1.0,1.0,0.0


,AEON DC-10T_BOX,AEON DC-1T,AEON DC-20T_BOX,AEON DC-3T
start_time,,,,
2020-11-09,1.0,0.0,2.0,0.0
2020-11-12,0.0,0.0,4.0,0.0
2020-11-16,0.0,0.0,3.0,1.0
2020-11-19,0.0,0.0,3.0,0.0
2020-11-23,1.0,0.0,5.0,1.0


,GCH DISTRIBUTION CENTRE- SEPANG-10T_BOX,GCH DISTRIBUTION CENTRE- SEPANG-1T,GCH DISTRIBUTION CENTRE- SEPANG-20T_BOX,GCH DISTRIBUTION CENTRE- SEPANG-3T
start_time,,,,
2020-11-06,1.0,0.0,5.0,1.0
2020-11-08,0.0,1.0,0.0,0.0
2020-11-09,1.0,0.0,4.0,1.0
2020-11-10,0.0,1.0,0.0,0.0
2020-11-11,1.0,0.0,10.0,0.0
2020-11-12,0.0,1.0,0.0,0.0
2020-11-15,0.0,1.0,0.0,0.0
2020-11-16,0.0,0.0,9.0,0.0
2020-11-17,0.0,1.0,0.0,0.0


,GCH RETAIL (MALAYSIA) SDN BHD-10T_BOX,GCH RETAIL (MALAYSIA) SDN BHD-1T,GCH RETAIL (MALAYSIA) SDN BHD-20T_BOX,GCH RETAIL (MALAYSIA) SDN BHD-3T
start_time,,,,
2020-11-08,1.0,0.0,0.0,0.0
2020-11-10,0.0,0.0,3.0,0.0
2020-11-12,0.0,0.0,2.0,0.0
2020-11-15,1.0,0.0,0.0,0.0
2020-11-17,0.0,0.0,4.0,0.0
2020-11-19,0.0,0.0,2.0,0.0
2020-11-22,0.0,0.0,0.0,1.0


,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC-10T_BOX,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC-1T,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC-20T_BOX,MYDIN CENTRAL DISTRIBUTION CENTRE - CDC-3T
start_time,,,,
2020-11-06,0.0,0.0,6.0,1.0
2020-11-10,0.0,1.0,0.0,0.0
2020-11-11,0.0,0.0,16.0,0.0
2020-11-12,0.0,1.0,2.0,0.0
2020-11-15,0.0,0.0,1.0,0.0
2020-11-16,0.0,0.0,1.0,1.0
2020-11-18,0.0,0.0,8.0,0.0
2020-11-19,0.0,0.0,9.0,0.0
2020-11-20,0.0,0.0,4.0,0.0


,WATSON-10T_BOX,WATSON-1T,WATSON-20T_BOX,WATSON-3T
start_time,,,,
2020-11-06,0.0,0.0,3.0,0.0
2020-11-09,0.0,0.0,5.0,0.0
2020-11-11,0.0,0.0,2.0,0.0
2020-11-16,0.0,0.0,11.0,0.0
2020-11-18,0.0,0.0,0.0,1.0
2020-11-20,0.0,0.0,4.0,0.0
2020-11-23,0.0,0.0,6.0,0.0


,TESCO DC(AMBIENT DC)-10T_BOX,TESCO DC(AMBIENT DC)-1T,TESCO DC(AMBIENT DC)-20T_BOX,TESCO DC(AMBIENT DC)-3T
start_time,,,,
2020-11-08,0.0,0.0,8.0,0.0
2020-11-12,0.0,0.0,15.0,0.0
2020-11-15,0.0,0.0,2.0,0.0
2020-11-17,0.0,0.0,7.0,0.0
2020-11-19,0.0,0.0,6.0,0.0
2020-11-20,0.0,0.0,1.0,0.0
2020-11-22,0.0,0.0,3.0,0.0


,JAYA GROCER DISTRIBUTION CENTER-10T_BOX,JAYA GROCER DISTRIBUTION CENTER-1T,JAYA GROCER DISTRIBUTION CENTER-20T_BOX,JAYA GROCER DISTRIBUTION CENTER-3T
start_time,,,,
2020-11-11,0.0,0.0,2.0,0.0
2020-11-13,-0.0,0.0,1.0,0.0
2020-11-17,-0.0,0.0,1.0,0.0
2020-11-19,0.0,0.0,2.0,0.0


In [20]:
output_imt

,AEON BIG (M) SDN. BHD - DC-10T_BOX,AEON BIG (M) SDN. BHD - DC-1T,AEON BIG (M) SDN. BHD - DC-20T_BOX,AEON BIG (M) SDN. BHD - DC-3T,AEON DC-10T_BOX,AEON DC-1T,AEON DC-20T_BOX,AEON DC-3T,GCH DISTRIBUTION CENTRE- SEPANG-10T_BOX,GCH DISTRIBUTION CENTRE- SEPANG-1T,...,WATSON-20T_BOX,WATSON-3T,TESCO DC(AMBIENT DC)-10T_BOX,TESCO DC(AMBIENT DC)-1T,TESCO DC(AMBIENT DC)-20T_BOX,TESCO DC(AMBIENT DC)-3T,JAYA GROCER DISTRIBUTION CENTER-10T_BOX,JAYA GROCER DISTRIBUTION CENTER-1T,JAYA GROCER DISTRIBUTION CENTER-20T_BOX,JAYA GROCER DISTRIBUTION CENTER-3T
start_time,,,,,,,,,,,,,,,,,,,,,
2020-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,...,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,0.0,0.0,8.0,0.0,NaN,NaN,NaN,NaN
2020-11-09,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,...,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-11,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,1.0,0.0,...,2.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0
2020-11-12,0.0,1.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,...,NaN,NaN,0.0,0.0,15.0,0.0,NaN,NaN,NaN,NaN
2020-11-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,0.0,1.0,0.0
2020-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,0.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN
2020-11-16,0.0,0.0,3.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,...,11.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DT (By Region)

## Preparing DT Data

In [25]:
# DT fleet forecast by customer
DT_O = df[df.Segment=='DT-OUTSTATION']
DT_L = df[df.Segment=='DT-LOCAL']
df_dt = DT_O.append(DT_L)

In [26]:
# Remove rows whose truck type is out of top 2 list
index_to_remove = []

for i in list(pd.value_counts(df_dt.FIRST_EQUIPMENT_GROUP_GID).index)[2:]: 
    index_to_remove+=list(df_dt[df_dt.FIRST_EQUIPMENT_GROUP_GID==i].index)
    
df_dt.drop(axis=0,labels=index_to_remove,inplace=True)

In [28]:
# DT ABT (Analytics Base Table)
df_dt_abt = abt(df_dt,"DestZone2")

## Load Pre-Trained Models

In [35]:
# Load pre-trained IMT models
DT_model = {}
for dest in DT_list:
    DT_model[dest] = pickle.load(open('models/dt_'+dest+'.pkl', 'rb'))

In [48]:
pd.to_datetime(datetime.today())

Timestamp('2020-11-25 14:23:54.003595')

In [36]:
DT_model

{'SOUTH': MultiOutputRegressor(estimator=GradientBoostingRegressor()),
 'NORTH': MultiOutputRegressor(estimator=GradientBoostingRegressor()),
 'CENTRAL': MultiOutputRegressor(estimator=GradientBoostingRegressor()),
 'EAST COAST': MultiOutputRegressor(estimator=RandomForestRegressor())}

In [38]:
# Use lastest few days as input for forecast 
dt_input = df_dt_abt[df_dt_abt.columns[:-2]].tail(number_of_day*4)

In [41]:
output_dt = pd.DataFrame(index=dt_input.index)
for region in list(set(dt_input.DestZone2)):
    try:
        temp_df = dt_input[dt_input.DestZone2==region]
        temp_df = temp_df[temp_df.columns[-4:]]
        temp_output = DT_model[region].predict(np.array(temp_df))

        temp_col_name = []
        for col in df_dt_abt.columns[-2:]:
            temp_col_name.append(region+"-"+col)
        temp_output = pd.DataFrame(np.round(temp_output),index=temp_df.index,columns=temp_col_name)
        temp_output.astype(np.int64)
        output_dt = output_dt.join(temp_output)
    except:
        pass

In [45]:
output_dt

,CENTRAL-10T_BOX,CENTRAL-20T_BOX,SOUTH-10T_BOX,SOUTH-20T_BOX,EAST COAST-10T_BOX,EAST COAST-20T_BOX,NORTH-10T_BOX,NORTH-20T_BOX
start_time,,,,,,,,
2020-11-13,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-14,1.0,-0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-15,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-16,1.0,0.0,1.0,8.0,1.0,3.0,0.0,4.0
2020-11-16,1.0,0.0,1.0,8.0,1.0,3.0,0.0,4.0
2020-11-16,1.0,0.0,1.0,8.0,1.0,3.0,0.0,4.0
2020-11-16,1.0,0.0,1.0,8.0,1.0,3.0,0.0,4.0
2020-11-17,3.0,15.0,1.0,1.0,1.0,9.0,3.0,6.0
2020-11-17,3.0,15.0,1.0,1.0,1.0,9.0,3.0,6.0
